In [81]:
import torch
from torch.nn import ReLU
import linear_model
from splade_model import SpladeModel

In [73]:
x = torch.tensor([[1, 2],
                  [1, 3],
                  [-2, 5]], dtype=torch.float64, requires_grad=True)

w = torch.tensor([[5, 4],
                  [1, 7]], dtype=torch.float64, requires_grad=True)

prod = x.mm(w)

max1, indice = torch.max(prod, 1)

max1.sum().backward()


In [74]:
print(prod)

print(max1)

print(x.grad)

print(w.grad)

tensor([[ 7., 18.],
        [ 8., 25.],
        [-5., 27.]], dtype=torch.float64, grad_fn=<MmBackward0>)
tensor([18., 25., 27.], dtype=torch.float64, grad_fn=<MaxBackward0>)
tensor([[4., 7.],
        [4., 7.],
        [4., 7.]], dtype=torch.float64)
tensor([[ 0.,  0.],
        [ 0., 10.]], dtype=torch.float64)


In [78]:
splade = SpladeModel.apply(x, w, None)

In [80]:
splade[0].sum().backward()

TypeError: SpladeModel.backward() takes 2 positional arguments but 3 were given

In [19]:
lm = linear_model.LinearModel.apply(x, w, None)

In [22]:
lm.sum().backward()

In [23]:
print(x.grad)

print(w.grad)

tensor([[ 9.,  8.],
        [14.,  9.],
        [13., 15.]], dtype=torch.float64)
tensor([[ 1., -2.],
        [13., 15.]], dtype=torch.float64)


In [137]:
x_int = torch.randint(1, 5, (3, 2, 2), dtype=torch.float64, requires_grad=True)
w_int = torch.randint(2, 4, (2, 4), dtype=torch.float64, requires_grad=True)


In [139]:
max1, indice = torch.max(x_int @ w_int, 1, keepdim=True)
print(max1)

print(x_int @ w_int)

tensor([[[18., 15., 18., 18.]],

        [[15., 14., 15., 15.]],

        [[15., 14., 15., 15.]]], dtype=torch.float64, grad_fn=<MaxBackward0>)
tensor([[[18., 15., 18., 18.],
         [15., 13., 15., 15.]],

        [[15., 11., 15., 15.],
         [15., 14., 15., 15.]],

        [[15., 14., 15., 15.],
         [ 6.,  5.,  6.,  6.]]], dtype=torch.float64,
       grad_fn=<UnsafeViewBackward0>)


In [88]:
max1.sum().backward()

In [97]:
print(indice)

torch.Size([3, 1, 4])


In [101]:
mask = torch.zeros_like(x_int @ w_int).scatter_(1, indice, 1.)
print(mask)

tensor([[[0., 0., 0., 0.],
         [1., 1., 1., 1.]],

        [[0., 0., 0., 0.],
         [1., 1., 1., 1.]],

        [[0., 0., 0., 0.],
         [1., 1., 1., 1.]]], dtype=torch.float64)


In [102]:
print(mask @ w_int.t())

tensor([[[ 0.,  0.],
         [11., 10.]],

        [[ 0.,  0.],
         [11., 10.]],

        [[ 0.,  0.],
         [11., 10.]]], dtype=torch.float64, grad_fn=<UnsafeViewBackward0>)


In [103]:
print(x_int.grad)
print(w_int.grad)

tensor([[[ 0.,  0.],
         [11., 10.]],

        [[ 0.,  0.],
         [11., 10.]],

        [[ 0.,  0.],
         [11., 10.]]], dtype=torch.float64)
tensor([[9., 9., 9., 9.],
        [8., 8., 8., 8.]], dtype=torch.float64)


In [49]:
torch.zeros_like(x_int).scatter_(1, indice, 1.)

tensor([[[1, 0],
         [0, 1]],

        [[1, 1],
         [0, 0]],

        [[1, 1],
         [0, 0]]])

In [122]:
x_int1 = torch.randint(-5, 4, (3, 4 ,6), dtype=torch.float64, requires_grad=True)
w_int1 = torch.randint(-5, 4, (6, 5), dtype=torch.float64, requires_grad=True)

In [123]:
m = ReLU()
rl = m(x_int1 @ w_int1)

In [124]:
rl1 = torch.max(x_int1 @ w_int1, torch.zeros_like(x_int1 @ w_int1))

In [125]:
rl.sum().backward()

In [126]:
print(rl)

tensor([[[ 0.,  0., 22., 19.,  9.],
         [ 0.,  4., 20., 20.,  5.],
         [ 0.,  0.,  3., 19., 21.],
         [ 0.,  0., 19., 10.,  5.]],

        [[10., 19.,  0.,  0.,  0.],
         [ 0.,  0., 19.,  8., 28.],
         [ 0.,  6., 21., 19., 12.],
         [17., 10., 15.,  0., 12.]],

        [[ 0.,  0.,  0., 19., 25.],
         [ 0.,  0.,  5., 10., 23.],
         [18.,  5.,  0.,  0.,  0.],
         [ 0.,  0., 24.,  3.,  1.]]], dtype=torch.float64,
       grad_fn=<ReluBackward0>)


In [131]:
mask = torch.zeros_like(rl)
mask[torch.nonzero(rl, as_tuple=True)] = 1

print(mask)

tensor([[[0., 0., 1., 1., 1.],
         [0., 1., 1., 1., 1.],
         [0., 0., 1., 1., 1.],
         [0., 0., 1., 1., 1.]],

        [[1., 1., 0., 0., 0.],
         [0., 0., 1., 1., 1.],
         [0., 1., 1., 1., 1.],
         [1., 1., 1., 0., 1.]],

        [[0., 0., 0., 1., 1.],
         [0., 0., 1., 1., 1.],
         [1., 1., 0., 0., 0.],
         [0., 0., 1., 1., 1.]]], dtype=torch.float64)


In [133]:
print(mask.size())

torch.Size([3, 4, 5])


In [132]:
print(mask @ w_int1.t())

tensor([[[ -3.,  -2.,   0., -11.,  -4.,  -8.],
         [ -1.,   0.,  -4., -11.,  -5., -11.],
         [ -3.,  -2.,   0., -11.,  -4.,  -8.],
         [ -3.,  -2.,   0., -11.,  -4.,  -8.]],

        [[  5.,   1.,  -6.,   1.,  -1.,  -3.],
         [ -3.,  -2.,   0., -11.,  -4.,  -8.],
         [ -1.,   0.,  -4., -11.,  -5., -11.],
         [  2.,  -1.,  -9.,  -5.,  -5.,  -7.]],

        [[ -1.,  -3.,   3.,  -8.,  -1.,  -6.],
         [ -3.,  -2.,   0., -11.,  -4.,  -8.],
         [  5.,   1.,  -6.,   1.,  -1.,  -3.],
         [ -3.,  -2.,   0., -11.,  -4.,  -8.]]], dtype=torch.float64,
       grad_fn=<UnsafeViewBackward0>)


In [127]:
print(x_int1.grad)

tensor([[[ -3.,  -2.,   0., -11.,  -4.,  -8.],
         [ -1.,   0.,  -4., -11.,  -5., -11.],
         [ -3.,  -2.,   0., -11.,  -4.,  -8.],
         [ -3.,  -2.,   0., -11.,  -4.,  -8.]],

        [[  5.,   1.,  -6.,   1.,  -1.,  -3.],
         [ -3.,  -2.,   0., -11.,  -4.,  -8.],
         [ -1.,   0.,  -4., -11.,  -5., -11.],
         [  2.,  -1.,  -9.,  -5.,  -5.,  -7.]],

        [[ -1.,  -3.,   3.,  -8.,  -1.,  -6.],
         [ -3.,  -2.,   0., -11.,  -4.,  -8.],
         [  5.,   1.,  -6.,   1.,  -1.,  -3.],
         [ -3.,  -2.,   0., -11.,  -4.,  -8.]]], dtype=torch.float64)


In [136]:
print(w_int1.grad)

tensor([[  7.,  -1., -20., -28., -25.],
        [ -4.,   0.,  -6.,  -8., -11.],
        [ -9.,  -9.,   2.,   8.,   4.],
        [  2.,  -1., -21., -19., -22.],
        [  2.,   2., -21., -16., -19.],
        [  2.,  -4.,   3.,  -2.,   1.]], dtype=torch.float64)


In [134]:
print(mask @ )

tensor([[[-73., -25.,  14., -62., -56.,   2.],
         [-74., -25.,   5., -63., -54.,  -2.],
         [-73., -25.,  14., -62., -56.,   2.],
         [-73., -25.,  14., -62., -56.,   2.]],

        [[  6.,  -4., -18.,   1.,   4.,  -2.],
         [-73., -25.,  14., -62., -56.,   2.],
         [-74., -25.,   5., -63., -54.,  -2.],
         [-39., -21., -12., -42., -36.,   2.]],

        [[-53., -19.,  12., -41., -35.,  -1.],
         [-73., -25.,  14., -62., -56.,   2.],
         [  6.,  -4., -18.,   1.,   4.,  -2.],
         [-73., -25.,  14., -62., -56.,   2.]]], dtype=torch.float64)
